In [ ]:
!python --version

Python 3.10.12


In [ ]:
!pip install datasets transformers accelerate torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import pandas as pd
import torch
from datasets import load_dataset, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, DataCollatorForSeq2Seq, EncoderDecoderModel
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import pipeline

In [ ]:
filetrain_path = "/content/drive/MyDrive/Summarizer_AI/Dataset/Fixed/train_df.csv"
filedev_path = "/content/drive/MyDrive/Summarizer_AI/Dataset/Fixed/dev_df.csv"

train_df = pd.read_csv(filetrain_path)
dev_df = pd.read_csv(filedev_path)

# Basic overview
print(train_df.shape)
print(train_df.info())
print(train_df.isnull().sum())

print(dev_df.shape)
print(dev_df.info())
print(dev_df.isnull().sum())

(36365, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36365 entries, 0 to 36364
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           36365 non-null  int64 
 1   original_text        36365 non-null  object
 2   abstractive_summary  36365 non-null  object
dtypes: int64(1), object(2)
memory usage: 852.4+ KB
None
Unnamed: 0             0
original_text          0
abstractive_summary    0
dtype: int64
(3624, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3624 entries, 0 to 3623
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           3624 non-null   int64 
 1   original_text        3624 non-null   object
 2   abstractive_summary  3624 non-null   object
dtypes: int64(1), object(2)
memory usage: 85.1+ KB
None
Unnamed: 0             0
original_text          0
abstractive_summary    0

In [ ]:
columns_to_delete = ['Unnamed: 0']

train_df = train_df.drop(columns=columns_to_delete)
dev_df = dev_df.drop(columns=columns_to_delete)

In [ ]:
train_df.iloc[1]

,1
original_text,liputan6 com jakarta menteri negara pembang...
abstractive_summary,menteri negara pembangunan nasional kwik kian ...


In [ ]:
dev_df.iloc[0]

,0
original_text,liputan6 com maluku gereja protestan di mal...
abstractive_summary,untuk menciptakan perdamaian di maluku gereja...


In [ ]:
import torch
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Device Name: {torch.cuda.get_device_name(0)}")

PyTorch Version: 2.5.1+cu121
CUDA Available: True
Device Name: Tesla T4


In [ ]:
device = 'cuda'

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "cahya/t5-base-indonesian-summarization-cased"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/793k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
dev_dataset = Dataset.from_pandas(dev_df)

In [ ]:
def preprocess_function_abstractive(examples):

    abstractive_summary = [
        " ".join(summary) if isinstance(summary, list) else summary
        for summary in examples["abstractive_summary"]
    ]  # List comprehension for handling list summaries

    # Tokenize the text input
    model_inputs = tokenizer(
        examples["original_text"], max_length=512, truncation=True, padding="max_length"
    )

    # Tokenize the combined summary (labels)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            abstractive_summary, max_length=256, truncation=True, padding="max_length"
        )

    # Add the tokenized labels to the model inputs
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_train_dataset = train_dataset.map(preprocess_function_abstractive, batched=True)
tokenized_dev_dataset = dev_dataset.map(preprocess_function_abstractive, batched=True)

Map:   0%|          | 0/36365 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/3624 [00:00<?, ? examples/s]

In [ ]:
print(train_dataset[0])
print(tokenized_train_dataset[0])

{'original_text': 'liputan6  com  jakarta  abu bakar ba asyir tidak kaget mengetahui majelis hakim menolak nota keberatan yang disampaikan tim pengacaranya di pengadilan negeri jakarta selatan  kamis  25 11    ya  biasa  itu sudah saya duga  kata ketua majelis mujahidin indonesia yang menjadi terdakwa kasus terorisme yang menerima keputusan hakim dengan wajah tanpa ekspresi  sebaliknya  tim pengacara pemimpin pondok pesantren al mukmin  ngruki  sukoharjo  jawa tengah  ini cukup kecewa dengan penolakan hakim itu  pasalnya  jaksa penuntut umum dianggap lebih banyak mengulang dakwaan yang sebelumnya sudah diselesaikan di pn jakarta pusat  mohamad assegaf  kuasa hukum ba asyir menyatakan dakwaan yang diajukan sangat dipaksakan dan tidak logis  dia mencontohkan  ba asyir dianggap memberikan inspirasi pada pengebom di depan kantor kedutaan besar australia di kawasan kuningan  jaksel  dan di hotel j  w  marriot  jaksel  padahal  saat kejadian  ba asyir berada dalam tahanan  karena itu mereka 

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./t5_summarization", #untuk menyimpan hasil
    num_train_epochs=3,  #berapa kali train
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    learning_rate=2e-5,
    gradient_accumulation_steps=4,
    evaluation_strategy="steps",
    eval_steps=500,
    save_total_limit=3,
    save_strategy="steps",
    save_steps=500,
    logging_dir='./logs',
    logging_steps=100,
    report_to="none",
    predict_with_generate=True,  # Tambahkan untuk evaluasi generasi teks
    generation_max_length=128,   # Panjang maksimal teks yang dihasilkan
    generation_num_beams=4       # Beam search untuk generasi teks
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_dev_dataset,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

In [ ]:
import os

# Define the path in Google Drive where you want to save the model
output_dir = '/content/drive/MyDrive/Summarizer_AI/Dataset/Hasil_FineTuning/T5:Abstractive'

# Create the directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save the trained model and tokenizer
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Model saved to {output_dir}")